In [1]:
import scipy
import matplotlib.pyplot as plt
from scipy.constants import c


k_dm = 1344.54095924  # Mhz/Tecu
seconds_to_microsecond = 1e6


import matplotlib.pyplot as plt
MPLSTYLE = '/arc/home/shiona/paper_sty.mplstyle'
plt.style.use(MPLSTYLE)

journal_textwidths = {'prd': 6.50127,
                      'none': 8.5,
                      'kaobook':6.5, # just a guess
                      'apj':6.0,
                      'aj': 6.0
                      } # Latex's \textwidth in inches

def figsize(fraction_of_textwidth = 0.45,height_per_width  = 1 / 1.61803399,journal = 'prd'):
    """When making your figure, decide its aspect ratio and your journal and its width as a fraction of the \textwidth of the journal. Then use matplotlib.pyplot.Figure(figsize = ...) to specify the right size"""
    textwidth = journal_textwidths[journal]
    width_in = textwidth * fraction_of_textwidth
    height_in = height_per_width * width_in
    return (width_in, height_in)

plt.rcParams["figure.figsize"] = figsize( 2 / 1.61803399,journal = 'apj')


In [2]:
#LBCS: longest baseline at 1890 km https://arxiv.org/pdf/2108.07284
LBCS,WFCS,chime={},{},{}
LBCS['baseline']=1890 #km
LBCS['freq']=144 #MHz
LBCS['name']='LBCS'

chime['baseline']=3300 #km
chime['freq']=600 #MHz
chime['name']='chime'

WFCS['baseline']=5000 #km not sure... https://iopscience.iop.org/article/10.3847/1538-3881/abc4e1/pdf
WFCS['freq']=4.3*10**3 #MHz
WFCS['name']='WFCS'



## These need to be double checked, probably wrong

In [68]:
#LBCS: longest baseline at 1890 km https://arxiv.org/pdf/2108.07284
LBCS,WFCS,chime={},{},{}
LBCS['baseline_high']=4 #km
LBCS['baseline_low']=1890 #km
LBCS['freq_high']=190 #MHz
LBCS['freq']=144 #MHz
LBCS['freq_low']=110 #MHz
LBCS['name']='LBCS'
LBCS['color']='darkblue'
LBCS['text_description']='100 mJy'
LBCS['text_description2']=r'$\delta >$0º'
LBCS['text_description3']='7500 square degrees'


chime['baseline_high']=3300 #km
chime['baseline_low']=66 #km
chime['freq_high']=800 #MHz
chime['freq']=600 #MHz
chime['freq_low']=400 #MHz
chime['name']='chime'
chime['color']='darkred'
chime['text_description']='500 mJy'
chime['text_description2']=r'$\delta >$-10º' #13,154 times beam size (pi*(1/60)**2) degrees=11.479030490366705º?
chime['text_description3']='94 square degrees' #13,154 times beam size (pi*(1/60)**2) degrees=11.479030490366705º?


WFCS['baseline_high']=5000 #km
WFCS['baseline_low']= 1000 #km
WFCS['freq_high']=2.4*10**3#MHz
WFCS['freq']=2.3*10**3 #MHz
WFCS['freq_low']=2.2*10**3 #MHz
WFCS['name']='WFCS'
WFCS['color']='gray'
WFCS['text_description']='150 mJy'
WFCS['text_description2']=r'$\delta >$-45º' #13,154 times beam size (np.pi*(1/60)**2) degrees=11.479030490366705º?
WFCS['text_description3']='11 square degrees' #13,154 times beam size (np.pi*(1/60)**2) degrees=11.479030490366705º?



In [69]:
from glob import glob
len(glob('/arc/projects/chime_frb/vlbi/revrfi_test2/*/*/*/369926569/calibrator_visibilities/*'))

445

In [70]:
frequency

array([1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05])

In [75]:
import numpy as np
fontsize=15
fontsize2=fontsize-3
figure = plt.figure(figsize=(5, 5))
grid = plt.GridSpec(1,1, figure=figure,hspace=.1)

axis_size=20

ax = plt.subplot(grid[0,0])
#ax1 = plt.subplot(grid[1,0])

catalogs=[LBCS,chime,WFCS]
for catalog in catalogs:
    catalog['lambda_low']=(c*1e-6/catalog['freq_high']) #m
    catalog['lambda_high']=(c*1e-6/catalog['freq_low'])
    conversion=180*3600/(np.pi)
    
    catalog['megalambda_high']=1/(catalog['baseline_high']*10**3/catalog['lambda_low'])*conversion
    catalog['megalambda_low']=1/(catalog['baseline_low']*10**3/catalog['lambda_high'])*conversion

    x_vals=[catalog['megalambda_high'],catalog['megalambda_low']]
    y_vals=[catalog['freq_high'],catalog['freq_low']]
    left=np.min(x_vals)
    right=np.max(x_vals)
    bottom=np.min(y_vals)
    width=np.abs(x_vals[0]-x_vals[1])
    height=np.abs(y_vals[0]-y_vals[1])
    alpha=0
    
    ax.scatter(x_vals[0],y_vals[0],color=catalog['color'],alpha=alpha)
    ax.scatter(x_vals[0],y_vals[1],color=catalog['color'],alpha=alpha)
    ax.scatter(x_vals[1],y_vals[0],color=catalog['color'],alpha=alpha)
    ax.scatter(x_vals[1],y_vals[1],color=catalog['color'],alpha=alpha)
    
    rect0=plt.Rectangle(xy=(left,bottom), width=width, height=height, fill=False,alpha=1,edgecolor='black')
    rect1=plt.Rectangle(xy=(left,bottom), width=width, height=height, facecolor=catalog['color'],label=catalog['name'],alpha=.5,edgecolor='black')

    r=catalog['text_description']
    center_x=10**np.mean(np.log10(x_vals))
    center_y=10**np.mean(np.log10(y_vals))
    if catalog['name']=='WFCS':
        center_y=max(y_vals)+max(y_vals)*.05
    ax.annotate(r, (center_x,center_y), color='black', weight='bold', 
                fontsize=fontsize2, ha='center', va='center')
    
    r=catalog['text_description2']
    center_y=center_y-.25*(10**np.max(np.log10(y_vals))-10**np.min(np.log10(y_vals)))
    if catalog['name']=='WFCS':
        center_y=center_y-2.5*(10**np.max(np.log10(y_vals))-10**np.min(np.log10(y_vals)))
    ax.annotate(r, (center_x,center_y), color='black', weight='bold', 
                fontsize=fontsize2, ha='center', va='center')
    
    ax.add_patch(rect0)
    ax.add_patch(rect1)
    
    
#ax.set_xlim(0,2)
ax.legend()

#ax.set_xlabel(r'$\frac{\lambda}{b} \rm{[arcseconds]}$',fontsize=fontsize)
ax.set_xticklabels([''])
ax.set_ylabel(r'$\nu \rm{[Mhz]}$',fontsize=fontsize)
ax.set_yscale('log')
ax.set_xscale('log')


frequency=ax.get_yticks()

ax2 = ax.twinx()
ax3 = ax.twiny()
ax2.set_yscale('log')
ylim=ax.get_ylim()
ax2.set_ylim(ylim)

ax3.set_xscale('log')
xlim=ax.get_xlim()
ax3.set_xlim(xlim)
ax3.set_xticklabels('')


# Match the ticks on the right y-axis with \(\lambda^2\)
#ticks=np.array([10**2,10**3])
#ax2.set_yticks(ticks)
#ax2.set_yticklabels((3*10**8/(ticks*10**6))**2)
ax2.set_yticklabels('')
ax.set_xlabel(r'$\frac{\lambda}{b} \rm{[arcseconds]}$',fontsize=fontsize)

#ax2.set_ylabel(r'$\lambda^2 [m]$',fontsize=fontsize)

figure.savefig('/arc/home/shiona/calibrator_surv/figures/garbage_phase_space.pdf',bbox_inches='tight')
#plt.show()


/tmp/ipykernel_721/4110596586.py:61: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''])


In [50]:
ax2.get_yticks(minor=False)

array([9.e+02, 9.e+00, 9.e-02])

In [29]:
1/(10**3*10**-6)

1000.0

In [21]:
import numpy as np
fontsize=15
fontsize2=fontsize-3
figure = plt.figure(figsize=(5, 10))
grid = plt.GridSpec(2,1, figure=figure,hspace=.1)

axis_size=20

ax = plt.subplot(grid[0,0])
ax1 = plt.subplot(grid[1,0])

catalogs=[LBCS,chime,WFCS]
for catalog in catalogs:
    catalog['lambda_low']=(c*1e-6/catalog['freq_high']) #m
    catalog['lambda_high']=(c*1e-6/catalog['freq_low'])
    conversion=180*3600/(np.pi)
    
    catalog['megalambda_high']=1/(catalog['baseline_high']*10**3/catalog['lambda_low'])*conversion
    catalog['megalambda_low']=1/(catalog['baseline_low']*10**3/catalog['lambda_high'])*conversion

    x_vals=[catalog['megalambda_high'],catalog['megalambda_low']]
    y_vals=[catalog['freq_high'],catalog['freq_low']]
    left=np.min(x_vals)
    right=np.max(x_vals)
    bottom=np.min(y_vals)
    width=np.abs(x_vals[0]-x_vals[1])
    height=np.abs(y_vals[0]-y_vals[1])
    alpha=0
    
    ax.scatter(x_vals[0],y_vals[0],color=catalog['color'],alpha=alpha)
    ax.scatter(x_vals[0],y_vals[1],color=catalog['color'],alpha=alpha)
    ax.scatter(x_vals[1],y_vals[0],color=catalog['color'],alpha=alpha)
    ax.scatter(x_vals[1],y_vals[1],color=catalog['color'],alpha=alpha)
    
    rect0=plt.Rectangle(xy=(left,bottom), width=width, height=height, fill=False,alpha=1,edgecolor='black')
    rect1=plt.Rectangle(xy=(left,bottom), width=width, height=height, facecolor=catalog['color'],label=catalog['name'],alpha=.5,edgecolor='black')

    r=catalog['text_description']
    center_x=10**np.mean(np.log10(x_vals))
    center_y=10**np.mean(np.log10(y_vals))
    if catalog['name']=='WFCS':
        center_y=max(y_vals)+max(y_vals)*.05
    ax.annotate(r, (center_x,center_y), color='black', weight='bold', 
                fontsize=fontsize2, ha='center', va='center')
    ax.add_patch(rect0)
    ax.add_patch(rect1)

#ax.set_xlim(0,2)
ax.legend()

#ax.set_xlabel(r'$\frac{\lambda}{b} \rm{[arcseconds]}$',fontsize=fontsize)
ax.set_xticklabels([''])
ax.set_ylabel(r'$\nu \rm{[Mhz]}$',fontsize=fontsize)
ax.set_yscale('log')
ax.set_xscale('log')



ax=ax1
catalogs=[LBCS,chime,WFCS]
for catalog in catalogs:
    x_vals=[catalog['megalambda_high'],catalog['megalambda_low']]
    y_vals=[catalog['lambda_high']**2,catalog['lambda_low']**2]
    left=np.min(x_vals)
    right=np.max(x_vals)
    bottom=np.min(y_vals)
    width=np.abs(x_vals[0]-x_vals[1])
    height=np.abs(y_vals[0]-y_vals[1])
    alpha=0
    ax1.scatter(x_vals[0],y_vals[0],color=catalog['color'],alpha=alpha)
    ax1.scatter(x_vals[0],y_vals[1],color=catalog['color'],alpha=alpha)
    ax1.scatter(x_vals[1],y_vals[0],color=catalog['color'],alpha=alpha)
    ax1.scatter(x_vals[1],y_vals[1],color=catalog['color'],alpha=alpha)
    
    rect0=plt.Rectangle(xy=(left,bottom), width=width, height=height, fill=False,alpha=1,edgecolor='black')
    rect1=plt.Rectangle(xy=(left,bottom), width=width, height=height, facecolor=catalog['color'],label=catalog['name'],alpha=.5,edgecolor='black')

    r=catalog['text_description2']
    center_x=10**np.mean(np.log10(x_vals))
    center_y=10**np.mean(np.log10(y_vals))
    if catalog['name']=='WFCS':
        center_y=max(y_vals)+max(y_vals)*.1
        center_x=max(x_vals)+max(x_vals)*.05
    ax1.annotate(r, (center_x,center_y), color='black', weight='bold', 
                fontsize=fontsize2, ha='center', va='center')
    ax1.add_patch(rect0)
    ax1.add_patch(rect1)
    

ax.set_xlabel(r'$\frac{\lambda}{b} \rm{[arcseconds]}$',fontsize=fontsize)
ax.set_ylabel(r'$\lambda^2 \rm{[m]}$',fontsize=fontsize)
ax.set_yscale('log')
ax.set_xscale('log')

figure.savefig('/arc/home/shiona/calibrator_surv/figures/garbage_phase_space.pdf',bbox_inches='tight')
plt.show()


/tmp/ipykernel_721/322239727.py:52: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([''])
